In [0]:
%sql
use catalog pavan_catalog_all


In [0]:
%sql
use schema bronze

In [0]:
# Example: Ingest raw data into Bronze
raw_df = (spark.read
    .format("json")
    .load("abfss://landing@parkaru15sa.dfs.core.windows.net/customers/")
)
display(raw_df)

In [0]:
raw_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("pavan_catalog_all.bronze.bronze_customers")

In [0]:
# %sql 
# drop table bronze.bronze_customers

In [0]:
display(
    spark.table("bronze_customers")
)

In [0]:
silver_df = spark.sql("""
    SELECT customer_id, customer_name, email
    FROM bronze.bronze_customers
    WHERE email IS NOT NULL
""")
display(silver_df)

In [0]:
silver_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("pavan_catalog_all.silver.silver_customers")

In [0]:
display(
    spark.table("silver.silver_customers")
)

In [0]:
gold_df = spark.sql("""
    SELECT customer_id as city, COUNT(*) as customer_count
    FROM silver.silver_customers
    GROUP BY customer_id
""")
display(gold_df)

In [0]:
gold_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("pavan_catalog_all.gold.gold_customers")

In [0]:
display(
    spark.table("gold.gold_customers")
)

In [0]:
%sql
-- Query Bronze
SELECT * FROM bronze.bronze_customers LIMIT 10;


In [0]:
%sql

-- Query Silver
SELECT * FROM silver.silver_customers LIMIT 10;

In [0]:
%sql


-- Query Gold
SELECT * FROM gold.gold_customers;


In [0]:
spark.sql("""
    CREATE OR REPLACE VIEW bronze.v_customers AS
    SELECT * FROM silver.silver_customers
""")

In [0]:
%sql
select * from bronze.v_customers

In [0]:
df_v_customers= spark.sql("""
    SELECT * FROM bronze.v_customers
""")
display(df_v_customers)

Databricks data profile. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
dbutils.data.summarize(df_v_customers)
